In [1]:
import pandas as pd
import psycopg2 
import os
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 10000

run  `ssh -L 5433:localhost:5432 cardano-cli@node-1.mainnet.ctl-runtime.staging.mlabs.city -N` 
to establish ssh tunnel


or to forward socket:
`mkdir /tmp/sql-socket`
`ssh -L /tmp/sql-socket/.s.PGSQL.5432:/run/postgresql/.s.PGSQL.5432 -N cardano-cli@node-1.mainnet.ctl-runtime.staging.mlabs.city`
`psql -h /tmp/sql-socket -U cexplorer -d cdbsync`


In [2]:
conn = psycopg2.connect(
    dbname='cdbsync',
    user='cexplorer',
    host='/tmp/sql-socket',
    port=5432
)

# conn.close()


In [3]:
all_tables_and_columns = """
SELECT
    t.tablename AS table_name,
    a.attname AS column_name
FROM
    pg_tables t
JOIN
    pg_attribute a ON t.tablename = a.attrelid::regclass::text
WHERE
    t.tableowner = 'cexplorer'
    AND a.attnum > 0
    AND NOT a.attisdropped
ORDER BY
    t.tablename,
    a.attnum;
"""
all_tables = pd.read_sql(all_tables_and_columns,conn)

/tmp/nix-shell.wU4exl/ipykernel_56345/3796777492.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_tables = pd.read_sql(all_tables_and_columns,conn)


In [4]:
all_tables

,table_name,column_name
0,ada_pots,id
1,ada_pots,slot_no
2,ada_pots,epoch_no
3,ada_pots,treasury
4,ada_pots,reserves
5,ada_pots,rewards
6,ada_pots,utxo
7,ada_pots,deposits
8,ada_pots,fees
9,ada_pots,block_id


In [3]:
max_time = """
SELECT max(time)
    
FROM
    block
;
"""
pd.read_sql(max_time,conn)

/tmp/nix-shell.6QLiFd/ipykernel_17949/478863478.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(max_time,conn)


,max
0,2022-12-31 11:13:48


In [3]:
blocks = """
SELECT
    *
FROM
    block
WHERE
  DATE(time)='2022-04-25'
LIMIT
    3;
"""
pd.read_sql(blocks,conn)

/tmp/nix-shell.wU4exl/ipykernel_56943/2541202552.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(blocks,conn)


,id,hash,epoch_no,slot_no,epoch_slot_no,block_no,previous_id,slot_leader_id,size,time,tx_count,proto_major,proto_minor,vrf_key,op_cert,op_cert_counter
0,7165446,"[b'\x0e', b'\xfa', b'y', b'q', b',', b'\x15', ...",334,59278513,353713,7165269,7165445,4981904,76849,2022-04-25 00:00:04,36,6,0,vrf_vk1rt4whnfaazzgxepp5vw0t59msuy8nyg9z6cadfd...,"[b'G', b'\xe2', b'>', b'\xe6', b'\x1b', b'\xfd...",7
1,7165447,"[b'\x94', b'\xc5', b'L', b'\x9c', b'\x84', b'\...",334,59278554,353754,7165270,7165446,7005759,4,2022-04-25 00:00:45,0,6,0,vrf_vk1t0syd45jwwwrfw6jpmc9668nqxmm4lq4wmjcfyq...,"[b'\xf5', b'\x8f', b'\xa6', b'\x1f', b'\x84', ...",0
2,7165448,"[b'\xea', b'.', b'\xf6', b'\xa3', b'I', b'\x8c...",334,59278567,353767,7165271,7165447,6096969,77679,2022-04-25 00:00:58,27,6,0,vrf_vk1v7n8q565aej4vqu3wkx5kcuxs3vslcr08pj70sa...,"[b'\x92', b'6', b'\xbb', b'\x96', b'\x9c', b'\...",6


In [36]:

blocks = """
SELECT
    ENCODE(tx.hash, 'hex') AS hash,
    block.block_no,
    block.time,
    block.tx_count,
    tx.script_size,
    tx.size
    
FROM block
LEFT JOIN tx ON block.id = tx.block_id    
WHERE
  DATE(block.time)='2022-04-25'

"""
df = pd.read_sql(blocks,conn)

/tmp/nix-shell.wU4exl/ipykernel_56943/4000378025.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(blocks,conn)


In [38]:
df.sort_values(by = 'size').head(2)


,hash,block_no,time,tx_count,script_size,size
77027,1ee30f055fc21ac067162b261bdb5034425bafb6e0b2e642554d0409c677c147,7168520,2022-04-25 19:30:44,32,0.0,190.0
44329,2075842a95a55d95d2bcb039dc1ad856f8fbe7cb73c68e60de1f5115b35310f7,7167412,2022-04-25 12:35:55,30,0.0,190.0


In [40]:
df.sort_values(by = 'script_size').head(2)

,hash,block_no,time,tx_count,script_size,size
0,f5d967e75124fcbb288e163a68f9da0c964afca2ff89ee9d5ffe38466afd9b92,7165269,2022-04-25 00:00:04,36,0.0,258.0
51491,86bac65004f7393808642027123cc0a3f74ed39b47f034e0a6b2b401dbf991fa,7167688,2022-04-25 14:19:57,31,0.0,876.0


In [33]:
df.to_pickle('df.pkl')

In [44]:

temp = """
SELECT
    ENCODE(tx.hash, 'hex') AS hash,
    tx.script_size,
    tx.size
    
    
FROM tx
WHERE script_size > 12000
LIMIT 10;
"""
pd.read_sql(temp,conn)

/tmp/nix-shell.wU4exl/ipykernel_56943/3580242147.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(temp,conn)


,hash,script_size,size
0,cb3828b8a1ecdfc269184548febddefa41fd582754b6b66d881865ba17288fdf,12997,14508
1,e3df8b42afccb3aacc83da65e0fb9b1907f095d50c8af940bea67cd67eb2dba0,13223,14459
2,347eb42d2546d56388b5d7390c6bb94da0117a488e8e70096751d68d89743506,13223,14459
3,70e805cbde9910a9a6f1d4f22326c31e6bba4452fb14d852c38f20236ff01f63,13223,14459
4,1ef46843cbdda3271893ab237e33016a9aa63c6eb9f0cd25e0495a221445ef66,13223,14459
5,224308825052ecf065ed5793461b4d55ba7e23eee2d2d5a8d916e96447707c87,13223,14711
6,fda591afa46f8d659a1d9e1691a1df0ed42f166898dc067c1bcb5e6a2331676d,13223,14711
7,0b3ebe7212f459790f9944dc3aaca3a3c1b3dd90d6027434f846c354cfd6f971,13223,14711
8,4992412407e7fbf7cbd2af87482ea718f4cab2f6318357351d13de5b89c54f4e,13223,14711
9,8f0695e1ed1b2d3a6a193a153be8b1badfb9e0f0e71e5648d1a3c809e40d0376,13223,14711


In [9]:

temp = """
SELECT
  ENCODE(hash, 'hex') AS hash,  
  -- ENCODE(bytes, 'hex') AS bytes,  
  *
    
FROM script
WHERE type='plutusV2'
LIMIT 10;

"""
pd.read_sql(temp,conn)

/tmp/nix-shell.6QLiFd/ipykernel_17949/1223304768.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(temp,conn)


,hash,id,tx_id,hash,type,json,bytes,serialised_size
0,c7ede8dab71e8b6195ca432705a42cc59ab33619e10399760b32f164,78310,54531089,"[b'\xc7', b'\xed', b'\xe8', b'\xda', b'\xb7', b'\x1e', b'\x8b', b'a', b'\x95', b'\xca', b'C', b""'"", b'\x05', b'\xa4', b',', b'\xc5', b'\x9a', b'\xb3', b'6', b'\x19', b'\xe1', b'\x03', b'\x99', b'v', b'\x0b', b'2', b'\xf1', b'd']",plutusV2,None,"[b'Y', b'\x18', b'W', b'\x01', b'\x00', b'\x00', b'3', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'2', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'""', b'""', b'#', b'#', b'%', b'3', b'S', b'3', b'\x00', b'c', b'\x00', b'\x80', b'\x05', b'0', b'\x07', b'\x00', b'C', b'3', b'5', b's', b'F', b'n', b'\x1c', b'\xd5', b'\\', b'\xea', b'\x80', b'\x12', b'@', b'\x00', b'F', b'd', b'B', b'F', b'`', b'\x02', b'\x00', b'`', b'\x04', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'f', b'f', b'\xae', b'h', b'\xcd', b'\xc3', b'\x9a', b'\xab', b'\x9d', b'P', b'\x0c', b'H', b'\x00', b'\x08', b'\xcc', b'\xcc', b'\xcc', b'\xcc', b'\xcc', ...]",6234
1,29ded634e7b2ea9d111e4775153b2c584ff309e75f4089c734ac1018,78311,54531102,"[b')', b'\xde', b'\xd6', b'4', b'\xe7', b'\xb2', b'\xea', b'\x9d', b'\x11', b'\x1e', b'G', b'u', b'\x15', b';', b',', b'X', b'O', b'\xf3', b'\t', b'\xe7', b'_', b'@', b'\x89', b'\xc7', b'4', b'\xac', b'\x10', b'\x18']",plutusV2,None,"[b'Y', b'\x18', b'V', b'\x01', b'\x00', b'\x00', b'3', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'2', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'""', b'""', b'#', b'#', b'%', b'3', b'S', b'3', b'\x00', b'c', b'\x00', b'\x80', b'\x05', b'0', b'\x07', b'\x00', b'C', b'3', b'5', b's', b'F', b'n', b'\x1c', b'\xd5', b'\\', b'\xea', b'\x80', b'\x12', b'@', b'\x00', b'F', b'd', b'B', b'F', b'`', b'\x02', b'\x00', b'`', b'\x04', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'f', b'f', b'\xae', b'h', b'\xcd', b'\xc3', b'\x9a', b'\xab', b'\x9d', b'P', b'\x0c', b'H', b'\x00', b'\x08', b'\xcc', b'\xcc', b'\xcc', b'\xcc', b'\xcc', ...]",6233
2,9704a5a1f944ae6ba27925689e3b187bfaaaaf7e486284fd23ffe4bb,78312,54531134,"[b'\x97', b'\x04', b'\xa5', b'\xa1', b'\xf9', b'D', b'\xae', b'k', b'\xa2', b'y', b'%', b'h', b'\x9e', b';', b'\x18', b'{', b'\xfa', b'\xaa', b'\xaf', b'~', b'H', b'b', b'\x84', b'\xfd', b'#', b'\xff', b'\xe4', b'\xbb']",plutusV2,None,"[b'Y', b'\x18', b'W', b'\x01', b'\x00', b'\x00', b'3', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'2', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'""', b'""', b'#', b'#', b'%', b'3', b'S', b'3', b'\x00', b'c', b'\x00', b'\x80', b'\x05', b'0', b'\x07', b'\x00', b'C', b'3', b'5', b's', b'F', b'n', b'\x1c', b'\xd5', b'\\', b'\xea', b'\x80', b'\x12', b'@', b'\x00', b'F', b'd', b'B', b'F', b'`', b'\x02', b'\x00', b'`', b'\x04', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'f', b'f', b'\xae', b'h', b'\xcd', b'\xc3', b'\x9a', b'\xab', b'\x9d', b'P', b'\x0c', b'H', b'\x00', b'\x08', b'\xcc', b'\xcc', b'\xcc', b'\xcc', b'\xcc', ...]",6234
3,af1280bc900a7a1824414ffb19b52d081c9f97a46202b7ea04e61b44,78313,54531274,"[b'\xaf', b'\x12', b'\x80', b'\xbc', b'\x90', b'\n', b'z', b'\x18', b'$', b'A', b'O', b'\xfb', b'\x19', b'\xb5', b'-', b'\x08', b'\x1c', b'\x9f', b'\x97', b'\xa4', b'b', b'\x02', b'\xb7', b'\xea', b'\x04', b'\xe6', b'\x1b', b'D']",plutusV2,None,"[b'Y', b'\x18', b'W', b'\x01', b'\x00', b'\x00', b'3', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'2', b'#', b'#', b'#', b'#', b'#', b'#', b'#', b'""', b'""', b'#', b'#', b'%', b'3', b'S', b'3', b'\x00', b'c', b'\x00', b'\x80', b'\x05', b'0', b'\x07', b'\x00', b'C', b'3', b'5', b's', b'F', b'n', b'\x1c', b'\xd5', b'\\', b'\xea', b'\x80', b'\x12', b'@', b'\x00', b'F', b'd', b'B', b'F', b'`', b'\x02', b'\x00', b'`', b'\x04', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'd', b'

In [10]:

temp = """
SELECT
  distinct type
    
FROM script



"""
pd.read_sql(temp,conn)

/tmp/nix-shell.6QLiFd/ipykernel_17949/911897974.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(temp,conn)


,type
0,plutusV2
1,plutusV1
2,timelock


In [13]:

temp = """
SELECT
  *
FROM tx_in
WHERE tx_id=54531089




"""
pd.read_sql(temp,conn)

/tmp/nix-shell.6QLiFd/ipykernel_17949/1922585346.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(temp,conn)


DatabaseError: Execution failed on sql '
SELECT
  *
FROM tx_in
WHERE tx_id=54531089




': column "tx_id" does not exist
LINE 5: WHERE tx_id=54531089
              ^
